In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torch import nn 
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
input_size= 28
sequence_length= 28
num_layers= 2
hidden_size= 256
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 3

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size= hidden_size
        self.num_layers= num_layers
        self.rnn= nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
        self.fc= nn.Linear(hidden_size * sequence_length, num_classes)
        
    
    def forward(self, x):
        h0= torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _= self.rnn(x, h0)
        out= out.reshape(out.shape[0], -1)
        out= self.fc(out)
        return out

In [6]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 53528523.48it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1564454.27it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 11861046.24it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1890120.92it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [7]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

In [8]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct=0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x= x.to(device= device).squeeze(1)
            y= y.to(device= device)
            
            scores= model(x)
            _, predictions= scores.max(1)
            num_correct +=(predictions==y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got{num_correct}/{num_samples} with accuracy:{float(num_correct)/float(num_samples)*100:.2f}')
        
    model.train()

In [9]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got58483/60000 with accuracy:97.47
Checking accuracy on test data
Got9693/10000 with accuracy:96.93
